# TP2 INFO834

Par RISS Ryan

Import des librairies :

In [ ]:
import pyarrow.compute as pc
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa

### Dataframes

In [140]:
villesDF = pd.read_csv('DATA/villes_virgule.csv')
academiesDF = pd.read_csv('DATA/academies_virgule.csv')

## Exercice 1

In [125]:
def fromDFToTable(df):
    return pa.Table.from_pandas(df)

In [126]:
def fromTableToDF(table):
    return table.to_pandas()

In [147]:
def fromTableToParquet(table, name):
    pq.write_table(table, name)
    return name

In [128]:
def fromParquetToTable(parquetName):
    return pq.read_table(parquetName)

### Tests

In [144]:
tableVilles = fromDFToTable(villesDF)
tableAcademies = fromDFToTable(academiesDF)

#print(tableVilles)
#print(tableAcademies)

In [145]:
newDfVilles = fromTableToDF(tableVilles)
newDfAcademies = fromTableToDF(tableAcademies)

#print(newDfVilles.head())
#print(newDfAcademies.head())

In [151]:
nameParquetVilles = fromTableToParquet(tableVilles, 'villes.parquet')
nameParquetAcademies = fromTableToParquet(tableAcademies, 'academies.parquet')

tbVilles = fromParquetToTable(nameParquetVilles)
tbAcademies = fromParquetToTable(nameParquetAcademies)

#print(tbVilles)
#print(tbAcademies)

## Exercice 2

In [ ]:
def printSchemaTable(table):
    df = table.to_pandas()
    print(df)

In [154]:
printSchemaTable(tableVilles)
printSchemaTable(tableAcademies)

## Exercice 3

In [62]:
def getColumnTable(table, column):
    return table.column(column)

In [156]:
getColumnTable(tableVilles, 'nom')

[
  [
    "Ozan",
    "Cormoranche-sur-Saône",
    "Plagne",
    "Tossiat",
    "Pouillat",
    ...
    "Sada",
    "Tsingoni",
    "Saint-Barthélemy",
    "Saint-Martin",
    "Saint-Pierre-et-Miquelon"
  ]
]

## Exercice 4

In [157]:
def getSumColumn(columnTable):
    return pc.sum(columnTable)

In [158]:
def getMinColumn(columnTable):
    return pc.min(columnTable)

In [159]:
def getMaxColumn(columnTable):
    return pc.max(columnTable)

In [160]:
def getCountColumn(columTable):
    return pc.count(columTable)

In [161]:
def getDistinctCountColumn(columnTable):
    return pc.count_distinct(columnTable)

In [162]:
column =  getColumnTable(tableVilles, 'nb_hab_2010')

print("Sum : " + str(getSumColumn(column)))
print("Min : " + str(getMinColumn(column)))
print("Max : " + str(getMaxColumn(column)))
print("Count : " + str(getCountColumn(column)))
print("Count distinct : " + str(getDistinctCountColumn(column)))

Sum : 64886015
Min : 0
Max : 2243833
Count : 36700
Count distinct : 5666


## Exercice 5

In [163]:
def informationCity(table, cityName):
    return table.filter(pa.compute.equal(table['nom'], cityName))

In [165]:
print(informationCity(tableVilles, 'Annecy'))

pyarrow.Table
dep: string
nom: string
cp: string
nb_hab_2010: int64
nb_hab_1999: int64
nb_hab_2012: int64
dens: int64
surf: double
long: double
lat: double
alt_min: double
alt_max: double
----
dep: [["74"]]
nom: [["Annecy"]]
cp: [["74000"]]
nb_hab_2010: [[50379]]
nb_hab_1999: [[50324]]
nb_hab_2012: [[50100]]
dens: [[3690]]
surf: [[13.65]]
long: [[6.11667]]
lat: [[45.9]]
...


In [166]:
def informationDepartment(table, department):
    res = table.filter(pa.compute.equal(table['dep'], department))
    df = res.to_pandas()
    sorted_df = df.sort_values(by='nom')
    sorted_table = fromDFToTable(sorted_df)
    return sorted_table

In [167]:
print(informationDepartment(tableVilles, '74'))

pyarrow.Table
dep: string
nom: string
cp: string
nb_hab_2010: int64
nb_hab_1999: int64
nb_hab_2012: int64
dens: int64
surf: double
long: double
lat: double
alt_min: double
alt_max: double
__index_level_0__: int64
----
dep: [["74","74","74","74","74",...,"74","74","74","74","74"]]
nom: [["Abondance","Alby-sur-Chéran","Alex","Allinges","Allonzier-la-Caille",...,"Épagny","Étercy","Étrembières","Évian-les-Bains","Évires"]]
cp: [["74360","74540","74290","74200","74350",...,"74330","74150","74100","74500","74570"]]
nb_hab_2010: [[1342,2012,985,3982,1637,...,3899,698,1887,8142,1294]]
nb_hab_1999: [[1297,1631,791,3022,1080,...,3233,630,1429,7278,1062]]
nb_hab_2012: [[1300,2000,1000,3800,1500,...,3800,700,1800,8100,1300]]
dens: [[22,306,57,265,170,...,575,153,347,1897,66]]
surf: [[58.84,6.56,17.02,15.01,9.62,...,6.77,4.55,5.43,4.29,19.49]]
long: [[6.73333,6.02084,6.23333,6.46667,6.11667,...,6.1,6,6.23333,6.58333,6.23333]]
lat: [[46.2833,45.8167,45.8833,46.3333,46,...,45.9333,45.8833,46.1833,46.

## Exercice 6

In [169]:
nbHab2012 =  getColumnTable(tableVilles, 'nb_hab_2012')
nombreMoyenHab2012 = getSumColumn(nbHab2012).as_py()/getCountColumn(nbHab2012).as_py()

print("Nombre moyen d'habitants en 2012 : " + str(nombreMoyenHab2012))

Nombre moyen d'habitants en 2012 : 1751.080272479564


In [168]:
dep74Info = informationDepartment(tableVilles, '74')

t = dep74Info.group_by("dep").aggregate([("nb_hab_2012", "sum"), ("nb_hab_2012", "count")])
t = t.to_pandas()
mean = (t.loc[0]["nb_hab_2012_sum"])/(t.loc[0]["nb_hab_2012_count"])
print("Nombre moyen d'habitants en 2012 pour le dep. 74 : " + str(mean))

Nombre moyen d'habitants en 2012 pour le dep. 74 : 2437.074829931973
